In [67]:
import sentencepiece as spm
import json
import pandas as pd

In [68]:
with open('openstack_lemma.json', 'r') as f:
    data = json.load(f)
df = pd.read_csv('../resource/openstack.csv', index_col=0, header=0)
df.set_index('commit_id', inplace=True)
input_list = []
for commit in data:
    if df.at[commit['commit_id'], 'strata'] != df['strata'].max():
        # input_list.append(' '.join(commit['text']))
        input_list.extend([txt for txt in commit['text']])
        


In [69]:
input_list[1]

'fix bug [ISSUEID].'

In [70]:
spm.SentencePieceTrainer.Train(
    sentence_iterator=iter(input_list),  # コーパスファイル
    model_type="unigram",  # デフォルト
    model_prefix='spm_openstack_3000_delstop_lemma',  # 出力されるモデルのファイル名に使われる
    vocab_size=3000,  # 語彙数
    pad_id = 3,
    character_coverage = 1.0,
    user_defined_symbols = ['[NUM]','[URL]','[COMMITID]','[CHANGEID]','[ISSUEID]'],
)

In [72]:
sp = spm.SentencePieceProcessor(model_file='spm_openstack_3000_delstop_lemma.model')

In [73]:
print(input_list[1])
encode = sp.Encode(input_list[1])
print(encode)
print(sp.encode(input_list[1], out_type=str))
decode = sp.Decode(encode)
print(decode)

fix bug [ISSUEID].
[18, 19, 9, 8, 10]
['▁fix', '▁bug', '▁', '[ISSUEID]', '.']
fix bug [ISSUEID].


In [ ]:
import pickle
import pandas as pd
with open('openstack_lemma.json', 'r') as f:
    data = json.load(f)
df = pd.read_csv('../resource/openstack.csv', index_col=0, header=0)
df.set_index('commit_id', inplace=True)
train_list = []
train_list2 = []
test_list = []
test_list2 = []
for commit in data:
    if df.at[commit['commit_id'], 'strata'] != df['strata'].max():
        train_list.append(' '.join(commit['text']))
        train_list2.append(int(df.at[commit['commit_id'], 'buggy']))
    else:
        test_list.append(' '.join(commit['text']))
        test_list2.append(int(df.at[commit['commit_id'], 'buggy']))
        
# train_data = [train_list, train_list2]
# test_data = [test_list, test_list2]
tr_df = pd.DataFrame(
    data={'data':train_list, 'label':train_list2},
    columns=['data', 'label']
)
te_df = pd.DataFrame(
    data={'data':test_list, 'label':test_list2},
    columns=['data', 'label']
)
tr_df.to_csv('openstack_train_delstop_lemma.csv')
te_df.to_csv('openstack_test_delstop_lemma.csv')

# with open('openstack_train.pkl', 'wb') as f_tr, open('openstack_test.pkl', 'wb') as f_te:
#     pickle.dump(train_data, f_tr)
#     pickle.dump(test_data, f_te)